In [1]:
# using Pkg; Pkg.add("DataFrames"); Pkg.add("CSV")

In [16]:
using CSV, DataFrames
df = CSV.read("iris.csv", footerskip=1,
            header=["PetalLength", "PetalWidth",
                    "SepalLength", "SepalWidth", "Class"]);


In [17]:
df.id = axes(df, 1);

In [18]:
sdf = stack(df)

,variable,value,Class,id
,Symbol,Float64⍰,String⍰,Int64
1,PetalLength,5.1,Iris-setosa,1
2,PetalLength,4.9,Iris-setosa,2
3,PetalLength,4.7,Iris-setosa,3
4,PetalLength,4.6,Iris-setosa,4
5,PetalLength,5.0,Iris-setosa,5
6,PetalLength,5.4,Iris-setosa,6
7,PetalLength,4.6,Iris-setosa,7
8,PetalLength,5.0,Iris-setosa,8
9,PetalLength,4.4,Iris-setosa,9


In [19]:
describe(sdf, :min, :max)

,variable,min,max
,Symbol,Any,Any
1,variable,PetalLength,SepalWidth
2,value,0.1,7.9
3,Class,Iris-setosa,Iris-virginica
4,id,1,150


In [20]:
udf = unstack(sdf, :variable, :value);

In [21]:
names(udf)

6-element Array{Symbol,1}:
 :Class      
 :id         
 :PetalLength
 :PetalWidth 
 :SepalLength
 :SepalWidth 

In [22]:
permutecols!(udf, names(df));

In [23]:
df == udf

true

In [24]:
using Statistics

In [25]:
agg = by(sdf, [:Class, :variable],
        x -> DataFrame(value=mean(x.value), n = nrow(x)))

,Class,variable,value,n
,String⍰,Symbol,Float64,Int64
1,Iris-setosa,PetalLength,5.006,50
2,Iris-versicolor,PetalLength,5.936,50
3,Iris-virginica,PetalLength,6.588,50
4,Iris-setosa,PetalWidth,3.418,50
5,Iris-versicolor,PetalWidth,2.77,50
6,Iris-virginica,PetalWidth,2.974,50
7,Iris-setosa,SepalLength,1.464,50
8,Iris-versicolor,SepalLength,4.26,50
9,Iris-virginica,SepalLength,5.552,50


In [26]:
agg2 = unstack(agg, :Class, :variable, :value);

In [27]:
agg3 = by(df, :Class) do x
    DataFrame(PetalLength=mean(x.PetalLength),
    PetalWidth=mean(x.PetalWidth),
    SepalLength=mean(x.SepalLength),
    SepalWidth=mean(x.SepalWidth));
end;

In [28]:
agg2 == agg3

true

In [29]:
agg2

,Class,PetalLength,PetalWidth,SepalLength,SepalWidth
,String⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,Iris-setosa,5.006,3.418,1.464,0.244
2,Iris-versicolor,5.936,2.77,4.26,1.326
3,Iris-virginica,6.588,2.974,5.552,2.026


In [30]:
agg3

,Class,PetalLength,PetalWidth,SepalLength,SepalWidth
,String⍰,Float64,Float64,Float64,Float64
1,Iris-setosa,5.006,3.418,1.464,0.244
2,Iris-versicolor,5.936,2.77,4.26,1.326
3,Iris-virginica,6.588,2.974,5.552,2.026
